# LSTM Model

In [ ]:


import pandas as pd
import ast
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from imblearn.over_sampling import SMOTE
from tensorflow.keras.layers import Input




# Load the dataset
df = pd.read_csv('/content/cleaned_dataset (1).csv')

# Display the first few rows to verify
print("Original DataFrame:")
print(df.head())
 # Use the 'tweet' column for text data
X = df['tweet']
# Use the 'class' column for labels
y = df['class']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Handle missing values in 'tweet' column
df['tweet'] = df['tweet'].fillna('')  # Replace NaN with empty strings



X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
# Use TfidfVectorizer for the 'tweet_tokens' column
tfidf = TfidfVectorizer()
X_tfidf = tfidf.fit_transform(df['tweet'])

# Encode target labels
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Handle class imbalance by converting labels to one-hot encoding
y_categorical = to_categorical(y_encoded)

Original DataFrame:
   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  \
0  retweet as a woman you should not complain abo...   
1  retweet boy dats coldtyga down bad for cuffin ...   
2  retweet dawg retweet you ever fuck a bitch and...   
3                     retweet she look like a tranny   
4  retweet the shit you hear about me might be tr...   

                                        tweet_tokens  
0  ['retweet', 'woman', 'complain', 'cleaning', '...  
1  ['retweet', 'boy', 'dat', 'coldtyga', 'bad', '...  
2  ['retweet', 'dawg', 'retweet', 'ever

In [ ]:
# Define the LSTM model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_tfidf.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(y_categorical.shape[1], activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001), metrics=['accuracy'])

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, min_delta=0.001)

# Sort the indices of the sparse matrix
X_tfidf.sort_indices()

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_tfidf, y_categorical, test_size=0.2, random_state=42)

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stopping],
                    verbose=1)

# Evaluate the model on the validation set
loss, accuracy = model.evaluate(X_val, y_val)
print(f'Validation accuracy: {accuracy:.3f}')

Epoch 1/10
620/620 [==============================] - 15s 22ms/step - loss: 0.4542 - accuracy: 0.8426 - val_loss: 0.3182 - val_accuracy: 0.8856
Epoch 2/10
620/620 [==============================] - 13s 20ms/step - loss: 0.2071 - accuracy: 0.9268 - val_loss: 0.3318 - val_accuracy: 0.8814
Epoch 3/10
620/620 [==============================] - 12s 20ms/step - loss: 0.1184 - accuracy: 0.9599 - val_loss: 0.4013 - val_accuracy: 0.8739
Epoch 4/10
620/620 [==============================] - 13s 20ms/step - loss: 0.0748 - accuracy: 0.9739 - val_loss: 0.4577 - val_accuracy: 0.8693
Epoch 5/10
620/620 [==============================] - 12s 19ms/step - loss: 0.0485 - accuracy: 0.9833 - val_loss: 0.5319 - val_accuracy: 0.8632
Epoch 6/10
155/155 [==============================] - 0s 2ms/step - loss: 0.5907 - accuracy: 0.8620
Validation accuracy: 0.862
